In [ ]:
from rich.pretty import Pretty

# Benchmark

ESS is built for various types of experiments with multiple instruments using different techniques.

Benchmarking was needed to make sure the data reduction workflow can handle the streaming data in real time.

## Benchmark Parameters

We are focusing on benchmarking the speed of computing process.

The baseline of the process speed is the pulse rate of 14/s.

The main parameters that affect to the speed are (1) size of data and (2) size of bins.

(1) size of data depends on to the event rate that an instrument receives
and (2) size of bins mostly depends on the number of pixels that each instrument(or a detector) has.

Therefore ``event rate`` and ``number of pixels`` are the main benchmarking parameters.

| parameter name   | corresponding quantity in the data reduction | simplified variable name |
| ---------------- | -------------------------------------------- | ------------------------ |
| event rate       | size of data                                 | event_rate               |
| number of pixels | size of bins                                 | num_pixels               |


Here is the list of benchmark parameters in the testing framework
and the default value of each parameters.

In [ ]:
from tests.prototypes.parameters import PrototypeParameters

Pretty(PrototypeParameters())

## Benchmark Tools

Benchmark tests and related tools are collected in ``tests/benchmarks`` and ``tests/prototypes``.

## ESS Instruments Requirements

Each instrument may have a range of requirements.

For example, an instrument can use different number of pixels depending on the experiment settings.

We have collected estimated numbers of event rate and number of pixels for each instrument.

In [ ]:
from tests.benchmarks.requirements import ESSInstruments

hw_requirements = ESSInstruments()
hw_requirements.show()